In [224]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/marketing-strategy-personalised-offer/sample.csv
/kaggle/input/marketing-strategy-personalised-offer/train_data.csv
/kaggle/input/marketing-strategy-personalised-offer/test_data.csv


### Importing modules and Dataset

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, OrdinalEncoder

### Minor EDA

In [226]:
train_data = pd.read_csv('../input/marketing-strategy-personalised-offer/train_data.csv')
train_data.head()

,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,Marital Status,restaurant type,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,...,restuarant_opposite_direction_house,has Children,visit restaurant with rating (avg),temperature,Restaur_spend_greater_than20,Travel Time,Climate,drop location,Prefer home food,Offer Accepted
0,2days,₹100000 or More,4~8,1,less1,Married partner,4 star restaurant,36,0,0,...,0,0,4,67,less1,22,Spring,Location B,0,No
1,2days,₹87500 - ₹99999,4~8,0,4~8,Married partner,Take-away restaurant,50plus,0,0,...,0,1,3,89,1~3,18,Summer,Location B,0,Yes
2,2days,₹87500 - ₹99999,less1,1,1~3,Single,Cold drinks,26,1,0,...,1,1,4,67,less1,7,Winter,Location A,1,Yes
3,10hours,₹37500 - ₹49999,less1,0,1~3,Single,Take-away restaurant,46,1,0,...,0,1,3,89,1~3,7,Summer,Location C,0,No
4,2days,₹100000 or More,never,1,1~3,Single,4 star restaurant,21,0,1,...,1,0,3,40,less1,7,Summer,Location C,0,No


In [227]:
train_data['travelled_more_than_5mins_for_offer'].unique()

array([1])

In [228]:
train_data[train_data['car'].notnull()]['Offer Accepted'].value_counts(),train_data['Offer Accepted'].value_counts()

(Yes    61
 No     50
 Name: Offer Accepted, dtype: int64,
 Yes    6994
 No     5385
 Name: Offer Accepted, dtype: int64)

In [229]:
train_data[['restuarant_same_direction_house','restuarant_opposite_direction_house']].value_counts()

restuarant_same_direction_house  restuarant_opposite_direction_house
0                                1                                      9345
1                                0                                      2240
                                 1                                       402
0                                0                                       392
dtype: int64

### Imputation

In [230]:
#missing values and unnecessary
all_col = [i for i in train_data.columns if i not in ["car","restuarant_opposite_direction_house","travelled_more_than_5mins_for_offer"]]

transformer1 = ColumnTransformer(
    [
        ("drop_cols", "drop", ["car","restuarant_opposite_direction_house","travelled_more_than_5mins_for_offer"]),
        ("imputer", SimpleImputer(strategy="most_frequent"), all_col)
    ],
    remainder="passthrough"
)

In [231]:
train_data1 = pd.DataFrame(transformer1.fit_transform(train_data), columns=all_col)

In [232]:
train_data1.columns

Index(['offer expiration', 'income_range', 'no_visited_Cold drinks',
       'travelled_more_than_15mins_for_offer', 'Restaur_spend_less_than20',
       'Marital Status', 'restaurant type', 'age',
       'Prefer western over chinese', 'travelled_more_than_25mins_for_offer',
       'no_visited_bars', 'gender', 'restuarant_same_direction_house',
       'Cooks regularly', 'Customer type', 'Qualification', 'is foodie',
       'no_Take-aways', 'Job/Job Industry', 'has Children',
       'visit restaurant with rating (avg)', 'temperature',
       'Restaur_spend_greater_than20', 'Travel Time', 'Climate',
       'drop location', 'Prefer home food', 'Offer Accepted'],
      dtype='object')

### Encoding

In [233]:
for i in train_data1.columns:
    if(len(train_data[i].unique())==2):
        print(i,': ',train_data[i].unique())

offer expiration :  ['2days' '10hours']
travelled_more_than_15mins_for_offer :  [1 0]
Prefer western over chinese :  [0 1]
travelled_more_than_25mins_for_offer :  [0 1]
gender :  ['Female' 'Male']
restuarant_same_direction_house :  [0 1]
Cooks regularly :  [1 0]
is foodie :  [0 1]
has Children :  [0 1]
Prefer home food :  [0 1]
Offer Accepted :  ['No' 'Yes']


In [234]:
no_encoding = ['travelled_more_than_15mins_for_offer','Prefer western over chinese','travelled_more_than_25mins_for_offer',
              'restuarant_same_direction_house','Cooks regularly','is foodie','has Children','Prefer home food','visit restaurant with rating (avg)']
len(no_encoding)

9

In [235]:
X_train_full = train_data1[no_encoding].astype('int64')
X_train_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12379 entries, 0 to 12378
Data columns (total 9 columns):
 #   Column                                Non-Null Count  Dtype
---  ------                                --------------  -----
 0   travelled_more_than_15mins_for_offer  12379 non-null  int64
 1   Prefer western over chinese           12379 non-null  int64
 2   travelled_more_than_25mins_for_offer  12379 non-null  int64
 3   restuarant_same_direction_house       12379 non-null  int64
 4   Cooks regularly                       12379 non-null  int64
 5   is foodie                             12379 non-null  int64
 6   has Children                          12379 non-null  int64
 7   Prefer home food                      12379 non-null  int64
 8   visit restaurant with rating (avg)    12379 non-null  int64
dtypes: int64(9)
memory usage: 870.5 KB


### Ordinal Variables

_There's only 'temperature' and 'travel time', which are numerical, though they could be considered ordinal variables  since they're discrete, and without much variance._

In [236]:
income_list = np.array(['Less than ₹12500', '₹12500 - ₹24999', '₹25000 - ₹37499', 
               '₹37500 - ₹49999', '₹50000 - ₹62499', '₹62500 - ₹74999', 
                '₹75000 - ₹87499', '₹87500 - ₹99999',  '₹100000 or More'])
no_list = np.array(['never', 'less1', '1~3', '4~8', 'gt8'])
offer_list = np.array(['10hours', '2days'])
age_list = np.array(['below21','21', '26', '31', '36', '41', '46','50plus'])
resto_list = np.array(['Cold drinks','Take-away restaurant', 'Restaurant with pub',
              '2 star restaurant','4 star restaurant'])
quali_list = np.array(['Some High School','High School Graduate', 'Some college - no degree','Associates degree',
              'Bachelors degree','Graduate degree (Masters or Doctorate)'])
custo_list = np.array(['Individual', 'With Colleagues', 'With Kids', 'With Family'])
marital_list = np.array([ 'Single', 'Unmarried partner','Married partner', 'Divorced', 'Widowed'])
temp_list = np.array([40, 67, 89])
travel_list = np.array([7, 10, 14, 18, 22])

### Nominal Variables

In [237]:
train_data1['Job/Job Industry'].unique()

array(['Unemployed', 'Arts Design Entertainment Sports & Media',
       'Sales & Related', 'Student', 'Business & Financial',
       'Computer & Mathematical', 'Office & Administrative Support',
       'Management', 'Healthcare Support', 'Life Physical Social Science',
       'Installation Maintenance & Repair', 'Legal',
       'Community & Social Services', 'Education&Training&Library',
       'Construction & Extraction',
       'Healthcare Practitioners & Technical',
       'Transportation & Material Moving', 'Retired',
       'Architecture & Engineering', 'Production Occupations',
       'Farming Fishing & Forestry', 'Protective Service',
       'Personal Care & Service', 'Food Preparation & Serving Related',
       'Building & Grounds Cleaning & Maintenance'], dtype=object)

In [238]:
train_data1['Climate'].unique()

array(['Spring', 'Summer', 'Winter'], dtype=object)

In [239]:
train_data1['drop location'].unique()

array(['Location B', 'Location A', 'Location C'], dtype=object)

In [240]:
income_list_oe = OrdinalEncoder(categories=[income_list],dtype=np.int64)
no_list_oe = OrdinalEncoder(categories=[no_list]*5,dtype=np.int64)
offer_list_oe = OrdinalEncoder(categories=[offer_list],dtype=np.int64)
age_list_oe = OrdinalEncoder(categories=[age_list],dtype=np.int64)
resto_list_oe = OrdinalEncoder(categories=[resto_list],dtype=np.int64)
quali_list_oe = OrdinalEncoder(categories=[quali_list],dtype=np.int64)
custo_list_oe = OrdinalEncoder(categories=[custo_list],dtype=np.int64)
marital_list_oe = OrdinalEncoder(categories=[marital_list],dtype=np.int64)
temp_list_oe = OrdinalEncoder(categories=[temp_list],dtype=np.int64)
travel_list_oe = OrdinalEncoder(categories=[travel_list],dtype=np.int64)
one_hot = OneHotEncoder(sparse=False,drop='first', dtype=np.int64)

In [241]:
train_data.columns

Index(['offer expiration', 'income_range', 'no_visited_Cold drinks',
       'travelled_more_than_15mins_for_offer', 'Restaur_spend_less_than20',
       'Marital Status', 'restaurant type', 'age',
       'Prefer western over chinese', 'travelled_more_than_25mins_for_offer',
       'travelled_more_than_5mins_for_offer', 'no_visited_bars', 'gender',
       'car', 'restuarant_same_direction_house', 'Cooks regularly',
       'Customer type', 'Qualification', 'is foodie', 'no_Take-aways',
       'Job/Job Industry', 'restuarant_opposite_direction_house',
       'has Children', 'visit restaurant with rating (avg)', 'temperature',
       'Restaur_spend_greater_than20', 'Travel Time', 'Climate',
       'drop location', 'Prefer home food', 'Offer Accepted'],
      dtype='object')

In [242]:
ord_enc = ['offer expiration','income_range','no_visited_Cold drinks','no_visited_bars','no_Take-aways',
           'Restaur_spend_less_than20','Restaur_spend_greater_than20', 'age','restaurant type','Qualification',
          'Customer type','Marital Status','temperature','Travel Time']

In [243]:
transformer2 = ColumnTransformer(
    [
        ('offer_list_oe',offer_list_oe,['offer expiration']),
        ('income_list_oe',income_list_oe,['income_range']),
        ('no_list_oe',no_list_oe,['no_visited_Cold drinks','no_visited_bars','no_Take-aways','Restaur_spend_less_than20','Restaur_spend_greater_than20']),
        ('age_list_oe',age_list_oe,['age']),
        ('resto_list_oe',resto_list_oe,['restaurant type']),
        ('quali_list_oe', quali_list_oe,['Qualification']),
        ('custo_list_oe',custo_list_oe,['Customer type']),
        ('marital_list_oe',marital_list_oe,['Marital Status']),
        ('temp_list_oe',temp_list_oe,['temperature']),
        ('travel_list_oe',travel_list_oe,['Travel Time'])
    ],
    remainder="drop"
)

In [244]:
ord_enc_data = pd.DataFrame(transformer2.fit_transform(train_data1), columns=ord_enc)
ord_enc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12379 entries, 0 to 12378
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   offer expiration              12379 non-null  int64
 1   income_range                  12379 non-null  int64
 2   no_visited_Cold drinks        12379 non-null  int64
 3   no_visited_bars               12379 non-null  int64
 4   no_Take-aways                 12379 non-null  int64
 5   Restaur_spend_less_than20     12379 non-null  int64
 6   Restaur_spend_greater_than20  12379 non-null  int64
 7   age                           12379 non-null  int64
 8   restaurant type               12379 non-null  int64
 9   Qualification                 12379 non-null  int64
 10  Customer type                 12379 non-null  int64
 11  Marital Status                12379 non-null  int64
 12  temperature                   12379 non-null  int64
 13  Travel Time                   1

In [245]:
transformer3 = ColumnTransformer(
    [
        ('one_hot1',one_hot,['Job/Job Industry']),
        ('one_hot2',one_hot,['Climate']),
        ('one_hot3',one_hot,['drop location']),
        ('one_hot4',one_hot,['gender'])
    ],
    remainder="drop"
)

In [247]:
one_hot_data = pd.DataFrame(transformer3.fit_transform(train_data1), columns=[name.split("__")[1] for name in transformer3.get_feature_names_out()])
one_hot_list = [name.split("__")[1] for name in transformer3.get_feature_names_out()]
one_hot_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12379 entries, 0 to 12378
Data columns (total 29 columns):
 #   Column                                                      Non-Null Count  Dtype
---  ------                                                      --------------  -----
 0   Job/Job Industry_Arts Design Entertainment Sports & Media   12379 non-null  int64
 1   Job/Job Industry_Building & Grounds Cleaning & Maintenance  12379 non-null  int64
 2   Job/Job Industry_Business & Financial                       12379 non-null  int64
 3   Job/Job Industry_Community & Social Services                12379 non-null  int64
 4   Job/Job Industry_Computer & Mathematical                    12379 non-null  int64
 5   Job/Job Industry_Construction & Extraction                  12379 non-null  int64
 6   Job/Job Industry_Education&Training&Library                 12379 non-null  int64
 7   Job/Job Industry_Farming Fishing & Forestry                 12379 non-null  int64
 8   Job/Job Industry

### Cumulation

In [248]:
X_train_full[one_hot_list] = one_hot_data
X_train_full[ord_enc] = ord_enc_data

In [249]:
X_train_full.shape

(12379, 52)

### Encoding the Label

In [250]:
le = LabelEncoder()

In [251]:
le.fit_transform(train_data['Offer Accepted'])

array([0, 1, 1, ..., 0, 1, 1])